## Prompt Classification Notebook

This notebook is meant to test how well the chatbot can perform the classification part of the pipeline. <br> <br>
DISCLAIMER: We have about `$`5.00 in free API credits and are charged by how many words are inputted and output. `$`5.00 is a lot of words, probably a thousands if not hundreds of thousands so we should be okay. But preferably look to query the chatbot at an upperbound of about ~500 queries. 

### Insert prompts

Update `PERSONA`, `CLASSIFICATION_PROMPT`, `DATA_REQ`, and `TEMPERATURE` below. 
* Notes:
    * `CLASSIFICATION_PROMPT` is a list of prompts. Add several prompts into the list to test them simultaneously, the output at the end will be for its respective prompt.
    * `DATA_REQ` defines the format you want GPT to respond in. It is appended to each classification prompt. 


In [1]:
# TODO UPDATE
# this is the identity of the bot, gives it some context for what it will do etc.
PERSONA: str = '''You are tasked with answering user queries for a hotel booking website. Your name is Nexus.'''

# TODO UPDATE
# this prompt is prepended to the users query everytime they send a message so dont make it too specific or generic.
CLASSIFICATION_PROMPT: list = [
'''Given the users prompt below classify what they are talking about into the following 4 categories: 
account query, hotel recommendation query, conversing, misc.''',

'''Using the users prompt below, you are tasked with determing what the user wants. Categorize their query as either an account query,
hotel related question, continuing a conversation with you, or unrelated/ misc''',

]

# TODO UPDATE
# This part is appended to every classification prompt, it shows what format the answer should be in
DATA_REQ = '''
If they are asking about something related to their account respond with:
{ type: ACC_QUERY }

If asking about adding a hotel to their watchlist, booking, or getting recommendations respond with. If the user does not provide enough information for you to fill out all of the data fields, then fill then in with NULL:
{ 
    type: HOTEL_RECC,
    data: {
        location: str,
        check_in: str,
        check_out: str,
        rooms: int,
        adults: int,
        children: int,
        radius: int,
        min_price: int,
        max_price: int,
    }
}

If they are continuing a conversation with you regarding booking a hotel, respond with. Update the data with what you knew previously
and what new information they give:
{ 
    type: CONVERSING 
    data: {
        location: str,
        check_in: str,
        check_out: str,
        rooms: int,
        adults: int,
        children: int,
        radius: int,
        min_price: int,
        max_price: int,
    }
}

If they are asking about something unrelated to the above respond with:
{ type: MISC }

Answer with these formats strictly and do not waver from it. If you are unsure about how to respond then respond with { type: MISC }.
'''

# TODO UPDATE
# this variable represents how "creative" gpts responses are. Values closer to 0 are more straightforward and values closer to 1 are more creative
# we probably want values closer the bot doesnt hallucinate but experiment.
TEMPERATURE: float = 0.2

### See prompts

Run the cell below to see what prompts, verbatim GPT will use.

In [2]:
print(f"Persona: {PERSONA}\n")

print(f"Temperature: {TEMPERATURE} \n")

classify_copy = CLASSIFICATION_PROMPT.copy()

print("Classification prompts that are going to be passed:\n")
for i in range(len(CLASSIFICATION_PROMPT)):
    CLASSIFICATION_PROMPT[i] += DATA_REQ
    print(f"Prompt {i}: {CLASSIFICATION_PROMPT[i]}\n")

Persona: You are tasked with answering user queries for a hotel booking website. Your name is Nexus.

Temperature: 0.2 

Classification prompts that are going to be passed:

Prompt 0: Given the users prompt below classify what they are talking about into the following 4 categories: 
account query, hotel recommendation query, conversing, misc.
If they are asking about something related to their account respond with:
{ type: ACC_QUERY }

If asking about adding a hotel to their watchlist, booking, or getting recommendations respond with. If the user does not provide enough information for you to fill out all of the data fields, then fill then in with NULL:
{ 
    type: HOTEL_RECC,
    data: {
        location: str,
        check_in: str,
        check_out: str,
        rooms: int,
        adults: int,
        children: int,
        radius: int,
        min_price: int,
        max_price: int,
    }
}

If they are continuing a conversation with you regarding booking a hotel, respond with. U

### Set-Up functions

Just run the cell below to set up the API request to GPT

In [3]:
from openai import OpenAI
import numpy as np
import pandas as pd
from IPython.display import display

client = OpenAI()
conversation = []
# initialize conversations with persona
for i in range(len(CLASSIFICATION_PROMPT)):
    conversation.append([{"role": "system", "content": PERSONA}])

def chat(prompt: str):
    global PERSONA, CLASSIFICATION_PROMPT, TEMPERATURE, client, conversation
    temp_responses = []
    
    for i in range(len(CLASSIFICATION_PROMPT)):
        prompt = CLASSIFICATION_PROMPT[i] + prompt
        conversation[i].append({"role": "user", "content": prompt})
        
        completion = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=conversation[i],
            temperature=TEMPERATURE,
        )
        response = completion.choices[0].message.content
        temp_responses.append(response)

        conversation[i].append({"role": "assistant", "content": response})

    return temp_responses

### Run 

Run the cell below to begin having a conversation with GPT. A textbox will a appear and a table will be made with GPTs outputs and which prompt was used for it. Type `quit` to end the conversation.

Criteria to judge outputs:
* Make sure if the user has typos in the prompt GPT fixes them in the response
* Make sure GPT follows the format (i.e. { type: , data: } format stricty. We converting GPT's response into a python variable so if it wavers python can't convert it to a variable
* Make sure it can get data when its presented in different ways. Example: Someone might want to book a room 2 people but might it say it like "book a room for me and my child" see if it can realize that means 1 adult, 1 child.

In [4]:
def main():
    user_prompt = ""
    while (True):
        user_prompt = input("Enter a prompt: ")
        if (user_prompt == "quit"):
            print("Chat ended")
            break
        print("")
        responses = chat(user_prompt)
        df = pd.DataFrame({
            "Prompt": classify_copy,
            "Response": responses,
        })
        df_styled = df.style.set_table_styles([
            {
                'selector': 'table',
                'props': [('width', '100%'), ('overflow', 'hidden')]  
            },
            {
                'selector': 'td',  
                'props': [
                    ('max-height', '100px'),
                    ('max-width', '650px'),
                    ('overflow', 'auto'), 
                    ('white-space', 'pre'), 
                    ('text-align', 'left'), 
                    ('padding', '5px'),     
                    ('border', '1px solid black') 
                ]
            }
        ])
        display(df_styled)
        print("")

main()

Enter a prompt:  Im looking to get a hotel this weekend can u help me?


,Prompt,Response
0,"Given the users prompt below classify what they are talking about into the following 4 categories: account query, hotel recommendation query, conversing, misc.","{ type: HOTEL_RECC, data: { location: NULL, check_in: NULL, check_out: NULL, rooms: NULL, adults: NULL, children: NULL, radius: NULL, min_price: NULL, max_price: NULL, } }"
1,"Using the users prompt below, you are tasked with determing what the user wants. Categorize their query as either an account query, hotel related question, continuing a conversation with you, or unrelated/ misc","{ type: HOTEL_RECC, data: { location: NULL, check_in: NULL, check_out: NULL, rooms: NULL, adults: NULL, children: NULL, radius: NULL, min_price: NULL, max_price: NULL, } }"


Enter a prompt:  I want one in fremont california for me and 1 child


,Prompt,Response
0,"Given the users prompt below classify what they are talking about into the following 4 categories: account query, hotel recommendation query, conversing, misc.","{ type: HOTEL_RECC, data: { location: ""Fremont, California"", check_in: NULL, check_out: NULL, rooms: 1, adults: 1, children: 1, radius: NULL, min_price: NULL, max_price: NULL, } }"
1,"Using the users prompt below, you are tasked with determing what the user wants. Categorize their query as either an account query, hotel related question, continuing a conversation with you, or unrelated/ misc","{ type: CONVERSING, data: { location: ""Fremont, California"", check_in: NULL, check_out: NULL, rooms: 1, adults: 1, children: 1, radius: NULL, min_price: NULL, max_price: NULL, } }"


Enter a prompt:  ill check in in a few days and want something cheap, not over $100


,Prompt,Response
0,"Given the users prompt below classify what they are talking about into the following 4 categories: account query, hotel recommendation query, conversing, misc.","{ type: CONVERSING data: { location: ""Fremont, California"", check_in: NULL, check_out: NULL, rooms: 1, adults: 1, children: 1, radius: NULL, min_price: NULL, max_price: 100, } }"
1,"Using the users prompt below, you are tasked with determing what the user wants. Categorize their query as either an account query, hotel related question, continuing a conversation with you, or unrelated/ misc","{ type: CONVERSING, data: { location: ""Fremont, California"", check_in: NULL, check_out: NULL, rooms: 1, adults: 1, children: 1, radius: NULL, min_price: NULL, max_price: 100, } }"


Enter a prompt:  quit


Chat ended
